# References

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from sec_certs.dataset.common_criteria import CCDataset
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

dset = CCDataset.from_web_latest()
certs_with_ids = {cert.heuristics.cert_id: cert for cert in dset if cert.heuristics.cert_id}

print(f"Certificates in dataset: {len(dset)}")
print(f"Certificates with extracted IDs: {len(certs_with_ids)}")

## Certificate report references

In [ ]:
refs_cr = nx.DiGraph()
for cert_id, cert in certs_with_ids.items():
    refs_cr.add_node(cert_id, cert=cert)
for cert_id, cert in certs_with_ids.items():
    if cr_refs := cert.heuristics.report_references.directly_referencing:
        for ref_id in cr_refs:
            refs_cr.add_edge(cert_id, ref_id, type=("cr",))
print(f"References in certificate reports: {len(refs_cr.edges)}")

## Security target references

In [ ]:
refs_st = nx.DiGraph()
for cert_id, cert in certs_with_ids.items():
    refs_st.add_node(cert_id, cert=cert)
for cert_id, cert in certs_with_ids.items():
    if st_refs := cert.heuristics.st_references.directly_referencing:
        for ref_id in st_refs:
            refs_st.add_edge(cert_id, ref_id, type=("st",))
print(f"References in security targets: {len(refs_st.edges)}")

## Combined references

In [ ]:
refs = nx.DiGraph()
refs.add_nodes_from(refs_cr)
refs.add_nodes_from(refs_st)
refs.add_edges_from(refs_cr.edges.data())
for edge in refs_st.edges:
    if edge in refs.edges:
        refs.edges[edge]["type"] = ("cr", "st")
    else:
        refs.add_edge(edge, *refs_st.edges[edge])
print(f"Combined references (not double counted): {len(refs.edges)}")

## Certificate overview
Enter the certificate you are interested in below and see its reference graph component.

In [ ]:
cert_id = "ANSSI-CC-2019/02"

In [ ]:
cert = certs_with_ids.get(cert_id)
if cert is None:
    print(f"Certificate with id {cert_id} is not present in the dataset.")

for component in nx.weakly_connected_components(refs):
    if cert_id in component:
        break

view = nx.subgraph_view(refs, lambda node: node in component)
print(f"Certificate with id {cert_id}:")
print(f" - is in a component with {len(view.nodes)} certificates and {len(view.edges)} references.")
print(f" - references {list(view[cert_id].keys())}")
print(f" - is referenced by {list(view.predecessors(cert_id))}")
print(f" - its page is at https://seccerts.org/cc/{cert.dgst}/")

In [ ]:
nx.draw(view, pos=nx.planar_layout(view), with_labels=True)